# MNIST en Deep Learning

Dans ce TP, nous allons construire des algorithmes de Deep Learning pour tenter de reconnaître des chiffres manuscrits.

## Chargement des données et transformation

Nous allons travailler sur la base de données MNIST qui contient 60000 images en niveaux de grille de résolution 28x28, représentant les 10 chiffres de 0 à 9, ainsi qu'un jeu de test de 10000 images. Tout d'abord, chargeons ce jeu de données.

In [ ]:
import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.ndimage
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics import Accuracy
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Les expériences ci dessous seront lancées sur {device}")

Téléchargement des données

In [ ]:
train_dataset = datasets.MNIST(
    "../data", train=True, transform=ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    "../data", train=False, transform=ToTensor(), download=True
)

Créer un dataloader et visualiser des images

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=16)


# Parcourez un batch dans le DataLoader
for images, labels in train_dataloader:
    print(
        "BATCH INFO : input shape: ",
        images.shape,
        " , labels shape: ",
        labels.shape,
        "\n",
    )
    sample_images = images[:5]  # Les 5 premières images
    sample_labels = labels[:5]  # Les 5 premières étiquettes
    break  # On prend seulement le premier batch

# Affichez les images et leurs étiquettes
fig, axes = plt.subplots(1, 5, figsize=(12, 3))

for i, ax in enumerate(axes):
    ax.imshow(
        sample_images[i].squeeze(), cmap="gray"
    )  # .squeeze() pour retirer la dimension du canal
    ax.set_title(f"Label: {sample_labels[i].item()}")
    ax.axis("off")

plt.tight_layout()
plt.show()

## Construire un premier réseau de neurones

Construisons notre premier réseau de neurones.

Pour cela, nous allons créer un modèle Pytorch en utilisant la classe `nn.Module` vu précedemment.

Puis utiliser les méthodes suivantes de Pytroch pour ajouter des couches à ce modèle :

* `nn.FLatten` : on manipule des vecteurs et non des image, on passe de (28,28) -> (784,)
* `nn.Linear` : on ajoute une couche dense (ou linéaire)
* `nn.Dropout` : applique un dropout à la couche, pour éviter le surapprentissage
* `nn.ReLU` : fonction d'activation relu au coeur du réseau
* `nn.Softmax` : en sortie de réseau

In [ ]:
# A COMPLETER

# Créer le réseau suivant avec Pytorch :
# 784 dimensions en entrée => 12 dense => activation relu => 12 dense => activation relu => dropout 0.5 => 10 sorties (activation softmax)


class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        # à compléter

    def forward(self, x):
        # à compléter
        return x


# Instanciation du modèle
model = MyModel()

# Déplace le modèle sur le device
model.to(device)

# affichage du résumé du modèle
summary(
    model, input_size=(1, 1, 28, 28)
)  # input_size= (batch_size, channels, dim_x, dim_y)

 On commence par créer une boucle d'entrainement qui va faciliter l'apprentissage de notre modèle sur nos données.

In [ ]:
# Défintion de la boucle d'entrainement
def train_loop(
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    optimizer: optim.SGD,
    epoch: int,
) -> torch.nn.Module:
    start_time = time.time()

    # Apprentissage du modèle
    model.train()  # Met le modèle en mode entraînement
    train_loss, count = 0.0, 0
    for X, y in tqdm(train_dataloader, desc=f"Train epoch {epoch}"):
        # Déplace les tenseurs sur le device
        X, y = X.to(device), y.to(device)
        # Prédiction du réseau de neurones
        y_hat = model(X)
        # Calcul de l'erreur (y_hat, y)
        loss = loss_fn(y_hat, y)
        # Backpropagation (MaJ des poids du réseau)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
        count += X.shape[0]
    train_loss /= count

    # Test du modèle
    accuracy = Accuracy(task="multiclass", num_classes=10).to(
        device
    )  # Définition de la métrique d'accuracy
    model.eval()
    with torch.no_grad():
        test_loss, count = 0.0, 0
        for X, y in tqdm(test_dataloader, desc=f"Test epoch {epoch}"):
            # Déplace les tenseurs sur le device
            X, y = X.to(device), y.to(device)

            # Prédiction du réseau de neurones
            y_hat = model(X)
            # Calcul de l'erreur (y_hat, y)
            loss = loss_fn(y_hat, y)
            test_loss += loss.item()

            accuracy.update(y_hat.argmax(dim=-1), y)
            count += X.shape[0]

    test_loss /= count
    elapsed_time = time.time() - start_time
    print(
        f"""
        --------------- Epoch {epoch} --------------- 
        Training Loss : {train_loss:.4f} | Test Loss : {test_loss:.4f} | Test Accuracy : {accuracy.compute():.2f} | Elapsed Time : {round(elapsed_time, 1)}s
        """
    )
    return model

On définit également les données nécessaires à l'apprentissage

In [ ]:
batch_size = 256
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(train_dataset, batch_size=batch_size)

Ensuite, nous pouvons lancer l'apprentissage des paramètres.

In [ ]:
model.parameters()

In [ ]:
epochs = 10
loss_fn = nn.CrossEntropyLoss()
sgd_optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, nesterov=True)
for epoch in range(epochs):
    model = train_loop(
        train_dataloader, test_dataloader, model, loss_fn, sgd_optimizer, epoch
    )

Nous vous laissons analyser les résultats. Ce réseau de neurones est-il performant ?

__A vous de jouer__ : essayez de créer un meilleur réseau de neurones, afin d'atteindre le meilleur résultat possible.

In [ ]:
# Créer un meilleur réseau de neurones, et l'entraîner
# Objectif : avoir le meilleur résultat possible

# Nous ne donnons pas la correction. Il y a plusieurs réponses possibles.
# Vous pouvez par exemple ajouter des couches, modifier le nombre de neurones par couche et jouer sur le dropout.

In [ ]:
epochs = 10
loss_fn = nn.CrossEntropyLoss()
sgd_optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, nesterov=True)
for epoch in range(epochs):
    model = train_loop(
        train_dataloader, test_dataloader, model, loss_fn, sgd_optimizer, epoch
    )

Voyons ce que donne notre modèle sur un exemple.

In [ ]:
def plot_prediction(model, dataloader):
    # Parcourez un batch dans le DataLoader
    for images, labels in dataloader:
        preds = model(images.to(device))
        sample_images = images[:5]  # Les 5 premières images
        sample_labels = labels[:5]  # Les 5 premières étiquettes
        sample_preds = torch.argmax(preds[:5], dim=1)  # Les 5 premières predictions
        break  # On prend seulement le premier batch

    # Affichez les images et leurs étiquettes
    fig, axes = plt.subplots(1, 5, figsize=(12, 3))

    for i, ax in enumerate(axes):
        ax.imshow(
            sample_images[i].squeeze(), cmap="gray"
        )  # .squeeze() pour retirer la dimension du canal
        ax.set_title(
            f"Label: {sample_labels[i].item()}, Pred: {sample_preds[i].item()}"
        )
        ax.axis("off")

    plt.tight_layout()
    plt.show()


plot_prediction(model, test_dataloader)

## CNN : réseaux de neurones convolutionnels

Nous allons maintenant implémenter un réseau de neurones convolutionnel.

Pour cet exercice, vous allez avoir besoin des méthodes Pytorch suivantes, en plus de celles déjà vues précédemment :

Construisons notre premier réseau de neurones.

* __nn.Conv2d__ : on ajoute une couche de convolution
* __nn.MaxPool2d__ : fonction de max pooling qui permet de réduire la dimension des images d'entrée

In [ ]:
# Créer le réseau suivant avec Pytorch :
# input_shape dimensions en entrée => 4 filtres de convolution 3x3 => activation relu =>
#    => maxpooling 2x2 => dropout 0.25 => flatten => 10 dense => dropout 0.5 =>
#    => activation relu => nb_classes sorties (activation softmax)

# Définition du modèle
class MyCNNModel(nn.Module):
    def __init__(self, input_shape, nb_classes):
        super().__init__()
        # à compléter

    def forward(self, x):
        # à compléter

        return x


# Instanciation du modèle
cnn_model = MyCNNModel()

# Déplace le modèle sur le device
cnn_model.to(device)

# affichage du résumé di modèle
summary(model, imput_size=())

In [ ]:
epochs = 10
loss_fn = nn.CrossEntropyLoss()
sgd_optimizer = optim.SGD(cnn_model.parameters(), lr=0.1, momentum=0.9, nesterov=True)
for epoch in range(epochs):
    cnn_model = train_loop(
        train_dataloader, test_dataloader, cnn_model, loss_fn, sgd_optimizer, epoch
    )

In [ ]:
plot_prediction(cnn_model, test_dataloader)

In [ ]:
# Créer un meilleur réseau de neurones convolutionnel, et l'entraîner
# Objectif : avoir le meilleur résultat possible

# Nous ne donnons pas la correction. Il y a plusieurs réponses possibles.
# Vous pouvez par exemple ajouter des couches convolutionnelles et max_pooling,
# modifier le nombre de convolutions ou leur taille et jouer sur le dropout.

In [ ]:
epochs = 10
loss_fn = nn.CrossEntropyLoss()
sgd_optimizer = optim.SGD(cnn_model.parameters(), lr=0.1, momentum=0.9, nesterov=True)
for epoch in range(epochs):
    cnn_model = train_loop(
        train_dataloader, test_dataloader, cnn_model, loss_fn, sgd_optimizer, epoch
    )